## AI 모델 Inference 성능 테스트

### [2024.07.11] 이상혁

이전에 깃허브에 업로드한 코드들은 AI 모델을 학습시킨 과정입니다.

**학습 과정**에서는 낚시성 영상과 아닌 영상의 자막을 Input 값으로 주고, 각각이 낚시성인지 아닌지를 정답으로 알려주는 방식으로 학습시켰습니다.

AI 모델은 정답을 잘 맞추는 방식으로 학습하면서 가중치를 업데이트하고, 해당 가중치를 저장해뒀습니다.

이제는 학습되어 있는 가중치를 load하여 Inference를 진행하겠습니다.

Inference는 **추론**입니다.

어느 정도 배운(학습한) 모델이 이제는 정답을 모르는 Input에 대해서 낚시성인지 아닌지 맞추는 과정입니다.

따라서 Inference 과정에서는 학습은 전혀 이뤄지지 않고, 정답만 맞추게 됩니다.

그리고 정답은 낚시성일 확률이 몇 퍼센트인지 알려주도록 설계하겠습니다.

### 0. 라이브러리

먼저 필요한 라이브러리를 import 합니다.

아래에 나와있는 라이브러리는 모두 필요하며, 없는 경우 커널 혹은 가상 환경에서 ```pip install {라이브러리 이름}```으로 설치하시면 됩니다.

버전은 따로 설정하지 않고 그냥 다운로드 받으면 됩니다.

In [36]:
# 라이브러리
import random

import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F

from transformers import AutoTokenizer, AutoModelForSequenceClassification, logging

from peft import get_peft_model, LoraConfig, TaskType

from caption_crawler import YouTubeCaptionCrawler # 직접 정의한 유튜브 자동생성자막 크롤러

이번에는 시드를 고정하겠습니다.

실제 서비스에서도 시드를 고정하면 하나의 입력값에 대해서 반복적으로 동일한 결과를 얻을 수 있습니다.

시드는 ```42```로 설정하겠습니다.

In [2]:
# 시드 고정
def fixSEED(seed, deterministic=True):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    if deterministic:
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

fixSEED(seed=42)

이번에는 GPU(cuda) 사용 가능 여부를 확인하겠습니다.

여기서 사용하는 모델은 BERT 기반으로 아키텍쳐를 구현했습니다.

파라미터 개수가 많은 모델이기 때문에 GPU가 없으면 구동할 수 없습니다.

In [3]:
# GPU 사용 여부 확인
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


아래의 함수는 GPU VRAM을 확인하는 코드입니다.

모델을 선언하는 코드를 여러 번 실행하면 GPU 메모리에 너무 많은 모델이 적재되어서 이후 Inference를 진행할 수 없습니다.

아래 함수를 수시로 실행하여 남은 GPU 메모리를 확인하면 됩니다.

In [5]:
# 학습 시작 전 GPU 메모리 확인
def checkGPU(empty=False):
    if torch.cuda.is_available():
        total_mem = torch.cuda.get_device_properties(0).total_memory
        current_mem = torch.cuda.memory_allocated(0)
        free_mem = total_mem - current_mem
        
        print(f'남은 GPU 메모리: {free_mem / (1024 ** 3):.2f} GB')
        
        # GPU 캐시에서 메모리 삭제
        if empty:
            torch.cuda.empty_cache()
            after_mem = torch.cuda.memory_allocated(0)
            print(f'GPU 캐시에서 메모리 삭제 후 메모리: {after_mem / (1024 ** 3):.2f} GB')
    else:
        print('CUDA 사용 불가')

In [6]:
checkGPU()

남은 GPU 메모리: 21.96 GB


### 1. 모델 정의 및 학습된 가중치 가져오기

먼저 제가 정의한 AI 모델입니다.

쉽게 생각하면 AI 모델 중에서 유명한 BERT 모델을 불러와서 **낚시성인지 아닌지를 분류하는 모델**을 만들었다고 생각하면 됩니다.

학습 시 사용한 모델 그 자체를 학습용 코드에서 가져왔습니다.

(아래 모델도 따로 ```.py```로 정의하고 import 하는게 일반적이지만 여기서는 가독성을 위해 그냥 첨부했습니다.)

In [7]:
class BaitingDetectBERT(nn.Module):
    def __init__(self, model_checkpoint, lora_r=8, lora_alpha=1, lora_dropout=0.1):
        super(BaitingDetectBERT, self).__init__()

        # Pre-trained Model
        self.model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=2)

        # LoRA Config
        lora_config = LoraConfig(
            task_type=TaskType.SEQ_CLS,
            target_modules=["query", 'key', "value"],  # Apply LoRA to the attention layers
            r=lora_r,
            lora_alpha=lora_alpha,
            lora_dropout=lora_dropout
        )

        # Apply LoRA to Pre-trained Layers
        self.model = get_peft_model(self.model, lora_config)

        # Dropout
        self.dropout = nn.Dropout(0.1)

    def forward(self, input_ids, attention_mask):
        outputs = self.model(input_ids=input_ids, attention_mask=attention_mask).logits
        outputs = self.dropout(outputs)
        return outputs

위에서 BERT 모델을 가져왔다고 했습니다.

BERT는 사전 학습된 모델입니다.

**사전 학습**이란 쉽게 생각해서 모델을 개발한 개발자가 모델이 여러 작업을 잘 수행할 수 있도록 많은 데이터로 학습시켜, 최적의 가중치를 세팅해두는 과정을 말합니다.

그리고 저는 사전 학습된 BERT가 낚시성 영상을 잘 분류하도록 ```BaitingDetectBERT```를 직접 설계해서 다시 한번 학습시킨 것입니다.

위 과정을 **Fine-tuning**이라고 합니다.

Backend 단에서는 굳이 모르셔도 좋습니다. 

사전 학습과 Fine-tuning과 관련된 설정은 아래에서 **모델 체크포인트**와 **가중치 경로**를 설정하게 됩니다.

모델 체크포인트는 개발자가 사전 학습시킨 모델의 이름, 가중치 경로는 제가 Fine-tuning한 모델의 가중치가 담겨있는 파일입니다.

In [8]:
# 모델 체크포인트
model_checkpoint = "klue/roberta-large"

# 모델 가중치 저장된 경로
model_path = 'YouTube(mixed)_RoBERTa_ACC_0.8731.pth'

### 2. 모델 Inference 파이프라인 설정

이제 사전학습한 모델 위에 파인튜닝한 가중치를 더해서 모델을 불러오고,

해당 모델이 새로운 영상에 대해서 몇 퍼센트 확률로 낚시성 영상인지 출력하도록 해보겠습니다.

이를 위해서 파이프라인을 설정하겠습니다.

자동생성자막 불러오기 -> 자동생성자막 토큰화(단어를 숫자로 매핑하는 과정) -> 모델에 가중치 덮어씌우기 -> 모델 Inference -> 확률 출력

위 과정을 수행하는 일련의 함수를 작성하겠습니다.

In [37]:
def bert_inference(model_checkpoint, model_path, link):
    '''
    Args
        model_checkpoint : 사전 학습 모델 이름
        model_path : Fine-tuning 모델 가중치가 저장된 경로
        link : 영상 링크
    '''
    
    # 모델 실행 전 사전 설정
    torch.cuda.empty_cache() # 메모리 캐시 삭제
    logging.set_verbosity_error() # 메시지 출력 off
    
    # GPU 사용여부 확인 후 지정
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    # 자동생성자막 불러오기
    caption = YouTubeCaptionCrawler(link).get_caption()
    
    # 토크나이저 정의(텍스트를 정수로 매핑시키는 데에 사용됩니다)
    tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
    
    # 모델 정의 및 가중치 덮어 씌우기
    model = BaitingDetectBERT(model_checkpoint) # 사전 학습 모델
    model.load_state_dict(torch.load(model_path)) # 파인 튜닝된 가중치 덮어 씌우기
    model.to(device)
    
    # 자동생성자막 토큰화
    inputs = tokenizer(caption, padding=True, truncation=True, return_tensors="pt") # 먼저 텍스트를 토큰화 후 정수로 매핑
    input_ids = inputs["input_ids"].to(device) # 정수 매핑
    attention_mask = inputs["attention_mask"].to(device) # 어텐션 마스크
    
    # 모델 추론
    model.eval()
    with torch.no_grad():
        logits = model(input_ids, attention_mask)
        probabilities = F.softmax(logits, dim=-1)[:, 1].item() # 낚시성일 확률
        
    # 결과 출력
    if probabilities >= 0.95:
        print(f'해당 영상은 {probabilities * 100:.2f}% 확률로 낚시성 요소가 포함되어 있습니다. 시청에 주의하세요.')
    elif probabilities >= 0.7:
        print(f'해당 영상은 {probabilities * 100:.2f}% 확률로 낚시성 요소가 포함되어 있습니다. 주의해주세요.')
    elif probabilities >= 0.5:
        print(f'해당 영상은 {probabilities * 100:.2f}% 확률로 낚시성 요소가 포함되어 있습니다. 약간 주의가 필요합니다.')
    elif probabilities >= 0.3:
        print(f'해당 영상은 {probabilities * 100:.2f}% 확률로 낚시성 요소가 포함되어 있습니다. 편하게 시청하세요.')
    else:
        print(f'해당 영상은 {probabilities * 100:.2f}% 확률로 낚시성 요소가 포함되어 있습니다. 재밌게 시청하세요.')
        
    return None

참고로 위 함수를 호출할 때마다 매번 모델을 불러오고 가중치를 덮어 씌우기 때문에 한번 작동하는데 오랜 시간이 걸립니다.

모델 정의 파트만 Backend 단에서 미리 정의해둘 수 있으면 그렇게 하는게 좋아보입니다.

### 3. 임의의 유튜브 영상으로 Inference

위에서 정의한 ```bert_inference``` 함수는 인자만 넣어주면 유튜브 영상에 대해서 분석해줍니다.

이번에는 임의의 영상을 가져와서 함수를 사용해보고, 각 영상의 낚시성 여부를 체크해보겠습니다.

#### 3-1. 게임 유튜버

첫번째 영상은 [링크](https://youtu.be/uJ4JUFMs91U?feature=shared)입니다.

제가 자주 보는 게임 유튜버로 게임에 대한 사실을 전달하는 유튜브입니다.

사실 여부가 중요하지는 않지만, 이런 경우에 가짜일 확률이 높게 나오지 않아야 하기 때문에 확인해보겠습니다.

학습 데이터에 해당 유튜버는 포함되어 있어서 아마 낚시성은 아니라고 판달할 것입니다.

In [32]:
# 첫번째
bert_inference(model_checkpoint, model_path, link = 'https://youtu.be/uJ4JUFMs91U?feature=shared')

해당 영상은 0.49% 확률로 낚시성 요소가 포함되어 있습니다. 재밌게 시청하세요.


#### 3-2. 낚시 유튜버

두번째 영상은 [링크](https://youtu.be/RYp6wQ7ep0g?feature=shared)입니다.

따로 낚시성 영상을 구하기 어려워서 유튜브에 '임영웅 결혼'을 검색했습니다.

학습 데이터에는 포함되지 않았지만 이런 류의 낚시성 영상이 많기 때문에 잘 검출하는지 확인해보겠습니다.

영상은 한번 클릭하셔서 보시면 왜 낚시성이라고 판단하는지 알 수 있습니다.

In [33]:
# 두번째
bert_inference(model_checkpoint, model_path, link = 'https://youtu.be/RYp6wQ7ep0g?feature=shared')

해당 영상은 98.99% 확률로 낚시성 요소가 포함되어 있습니다. 시청에 주의하세요.


#### 3-3. 정육점 유튜버

세번째 영상은 [링크](https://youtu.be/6AkrK9Xo0DM?feature=shared)입니다.

정육점 관련 영상입니다. 학습 데이터에 포함되어 있지 않은 유튜버로 기억합니다.

정육 관련 정보를 전달하는 훌륭한 유튜버입니다.

학습 데이터가 아닌 유튜버에 대해서도 올바르게 Inference 하는지 확인해보겠습니다.

In [34]:
# 세번째
bert_inference(model_checkpoint, model_path, link = 'https://youtu.be/6AkrK9Xo0DM?feature=shared')

해당 영상은 0.37% 확률로 낚시성 요소가 포함되어 있습니다. 재밌게 시청하세요.


#### 3-4. 다큐 유튜버

네번째 영상은 [링크](https://youtu.be/Gz7gBnEnXlI?feature=shared)입니다.

EBS 다큐멘터리입니다. 이런 경우도 낚시성이라고 나오지 않는게 바람직합니다.

수치가 높게 나오지는 않지만 그래도 20% 정도 나오는게 의아합니다.
- 학습 데이터를 추가
- 카테고리나 공신력 있는 유튜버를 수집해서 모델이 처리하기 전에 '공신력 있는 매체 혹은 유튜버입니다'라고 알려주는 것도 좋은 방법이라고 생각합니다. 
- 위와 같은 방법을 **사전 질의 차단**이라고 부르겠습니다.

In [38]:
# 네번째
bert_inference(model_checkpoint, model_path, link = 'https://youtu.be/Gz7gBnEnXlI?feature=shared')

해당 영상은 18.78% 확률로 낚시성 요소가 포함되어 있습니다. 재밌게 시청하세요.


#### 3-5. 예능 유튜버

다섯번째 영상은 [링크](https://youtu.be/ZHc5r2udNC8?feature=shared)입니다.

이번에는 예능입니다. 예능 검색해서 나오는 영상을 사용해보겠습니다.

예능은 낚시성이냐 비낚시성이냐가 중요한 분야가 아니라고 생각합니다.

그래도 예능에 대해서도 비낚시성이라고 말하는 것이 일반적이라고 생각합니다.

모델은 어떻게 판단하는지 확인해보겠습니다.

In [39]:
# 다섯번째
bert_inference(model_checkpoint, model_path, link = 'https://youtu.be/ZHc5r2udNC8?feature=shared')

해당 영상은 0.39% 확률로 낚시성 요소가 포함되어 있습니다. 재밌게 시청하세요.


#### 3-6. 음악방송 유튜버

여섯번째 영상은 [링크](https://youtu.be/ulMzXgPmPhY?feature=shared)입니다.

이번에는 음악방송입니다. 이 경우도 **사전에 질의를 차단**하는 것이 적합하다고 생각합니다.

모델은 음악방송에 대해서 어떻게 판단하는지 궁금해서 시도해봤습니다.

In [40]:
# 여섯번째
bert_inference(model_checkpoint, model_path, link = 'https://youtu.be/ulMzXgPmPhY?feature=shared')

해당 영상은 3.47% 확률로 낚시성 요소가 포함되어 있습니다. 재밌게 시청하세요.


#### 3-7. 애매한 유튜버

일곱번째 영상은 [링크](https://youtu.be/UE13lU7MrSk?feature=shared)입니다.

해당 영상은 국뽕 유튜버에 대해서 설명하는 지식 전달 채널입니다.

물론 사실 검증은 안되지만, 낚시성 영상들과는 거리가 멀다고 생각합니다.

모델도 그렇게 판단할지 확인하겠습니다.

이 경우에는 낚시성이라고 판단하고 있습니다. 이 부분은 개선이 필요해보입니다.
- 정확한 구분으로는 낚시성 유튜버는 아니지만 애매한 유튜버를 해당 유튜버 같은 케이스로 설정할 수 있을 것 같습니다.
- 비슷한 경우에 대해서 더 수집해야 합니다.

In [41]:
# 일곱번째
bert_inference(model_checkpoint, model_path, link = 'https://youtu.be/UE13lU7MrSk?feature=shared')

해당 영상은 75.30% 확률로 낚시성 요소가 포함되어 있습니다. 주의해주세요.


### 4. 마무리

간단하게 몇가지 경우만 살펴봤습니다.

전반적으로 개선점이 보입니다.

1. Backend 단계에서 모델 정의 부분을 분리할 것인가?
2. 라이브 영상의 경우 video id를 추출하는 로직을 추가해야 한다.
3. 7번째 영상과 같은 애매한 경우를 더 찾아서 수집하자.
4. 확률에 따라서 메시지를 어떻게 다르게 할 것인가?

❗️따라서 가짜/애매/진짜로 나눠서 유튜브 데이터를 더 수집해야 합니다.❗️

다들 시간 날 때마다 많이 추가해주세요.

코드에 대해서 궁금하면 바로바로 알려주세요!